<h1 style="display:inline-block">ActiveVoxel Design</h1><br/>

subdivide( get_faces( solid ), divisions )

In [9]:
# cube_vxs = [[ px py pz invp id demi? ] ...]

# invp is the level of division of the unitary space
#  (the inverse of the line parameter in parametric form l = p.t + c)
# p in px, py, pz is a multiplier 0 <= p <= invp

In [10]:
import numpy as np

vxs_py = [[ 1, 0, 0, 1, 0, 0 ], [ 0, 0, 0, 1, 1, 0 ], [ 0, 0, 1, 1, 2, 0 ], [ 1, 0, 1, 1, 3, 0 ], 
          [ 0, 1, 1, 1, 4, 0 ], [ 1, 1, 1, 1, 5, 0 ], [ 0, 1, 0, 1, 6, 0 ], [ 1, 1, 0, 1, 7, 0 ]]

vxs = np.array( vxs_py )

print( vxs.transpose() )


[[1 0 0 1 0 1 0 1]
 [0 0 0 0 1 1 1 1]
 [0 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1]
 [0 1 2 3 4 5 6 7]
 [0 0 0 0 0 0 0 0]]


In [11]:
# deriving vxs unique ids from their positions
# tmax is the max( invp ) - calling invp = t
tmax = 1

vx_names = { ( tmax + 1 ) * ( tmax + 1 ) * vx[ 0 ] + ( tmax + 1 ) * vx[ 1 ] + vx[ 2 ]: 
             ( vx[ 3 ], vx[ 0 ], vx[ 1 ], vx[ 2 ], tmax ) for vx in vxs_py }

print( vx_names )


{4: (1, 1, 0, 0, 1), 0: (1, 0, 0, 0, 1), 1: (1, 0, 0, 1, 1), 5: (1, 1, 0, 1, 1), 3: (1, 0, 1, 1, 1), 7: (1, 1, 1, 1, 1), 2: (1, 0, 1, 0, 1), 6: (1, 1, 1, 0, 1)}


In [12]:
# edges divided in metadata and data

# vx1 vx2 id parent

edges_meta_py = [[ 0, 1, 1, 0 ], [ 1, 2, 2, 0 ], [ 2, 3, 3, 0 ], [ 3, 0, 4, 0 ], [ 2, 4, 5, 0 ], 
                 [ 4, 5, 6, 0 ], [ 5, 3, 7, 0 ], [ 4, 6, 8, 0 ], [ 6, 7, 9, 0 ], [ 7, 5, 10, 0 ],
                 [ 6, 1, 11, 0 ], [ 0, 7, 12, 0 ]]


edges_meta = np.array( edges_meta_py )

max_edge = np.max( edges_meta, 0 )[ 2 ]

print( max_edge )

print( edges_meta.transpose())


12
[[ 0  1  2  3  2  4  5  4  6  7  6  0]
 [ 1  2  3  0  4  5  3  6  7  5  1  7]
 [ 1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]


In [13]:
# dx dy dz divs1 chdiv1 div2 chdiv2 id
# ( dx, dy, dz ) = direction vector = vx2 - vx1
# divs[n] = how many times this edge were divided related to face n
# chdiv[n] = children related to face n were created dividing this edge chdiv[n] times


edt_py = [[ vxs_py[ id[ 0 ]][ 0 ], 
            vxs_py[ id[ 0 ]][ 1 ], 
            vxs_py[ id[ 0 ]][ 2 ], 1, 0, 1, 0, id[ 2 ]] 
              for id in edges_meta_py ]

edges_data = np.array( edt_py )

edt_py2 = [[ vxs_py[ id[ 1 ]][ 0 ], 
            vxs_py[ id[ 1 ]][ 1 ], 
            vxs_py[ id[ 1 ]][ 2 ], 0, 0, 0, 0, 0 ] 
              for id in edges_meta_py ]

ed2 = np.array( edt_py2 )

edges_data -= ed2
edt_py = edges_data.tolist()

print( edt_py )
print()
print( edges_data.transpose())


[[1, 0, 0, 1, 0, 1, 0, 1], [0, 0, -1, 1, 0, 1, 0, 2], [-1, 0, 0, 1, 0, 1, 0, 3], [0, 0, 1, 1, 0, 1, 0, 4], [0, -1, 0, 1, 0, 1, 0, 5], [-1, 0, 0, 1, 0, 1, 0, 6], [0, 1, 0, 1, 0, 1, 0, 7], [0, 0, 1, 1, 0, 1, 0, 8], [-1, 0, 0, 1, 0, 1, 0, 9], [0, 0, -1, 1, 0, 1, 0, 10], [0, 1, 0, 1, 0, 1, 0, 11], [0, -1, 0, 1, 0, 1, 0, 12]]

[[ 1  0 -1  0  0 -1  0  0 -1  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  1 -1]
 [ 0 -1  0  1  0  0  0  1  0 -1  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  2  3  4  5  6  7  8  9 10 11 12]]


In [14]:
# edge1 edge2 id
# id < 0 = reversed edge

tris_py = [[  1,  2, 0 ], [  3,  4, 1 ], [  -3,  5, 2 ], [  6,   7, 3 ], [  -6,  8,  4 ], [  9, 10,  5 ],
           [ -9, 11, 6 ], [ -1, 12, 7 ], [ -12, -4, 8 ], [ -7, -10, 9 ], [ -11, -8, 10 ], [ -5, -2, 11 ]]

tris = np.array( tris_py )
max_tri = np.max( tris, 0 )[ 2 ]

print( tris.transpose())


[[  1   3  -3   6  -6   9  -9  -1 -12  -7 -11  -5]
 [  2   4   5   7   8  10  11  12  -4 -10  -8  -2]
 [  0   1   2   3   4   5   6   7   8   9  10  11]]


In [15]:
# tri1 tri2 id
# we do not keep face divisions because unlike edges, parent faces (faces that we divide and generate child faces)
# are immediately erased and replaced by its children

faces_py = [[ 0, 1, 0 ], [ 2, 3, 1 ], [ 4, 5, 2 ], [ 6, 7, 3 ], [ 8, 9, 4 ], [ 10, 11, 5 ]]

faces = np.array( faces_py )

max_face = np.max( faces, 0 )[ 2 ]

print( faces.transpose())


[[ 0  2  4  6  8 10]
 [ 1  3  5  7  9 11]
 [ 0  1  2  3  4  5]]


In [16]:
def subdivide( _faces, divs ):
    global max_tri, max_face, faces
    
    nnewf = divs * divs
    nnewt = nnewf * 2
    
    newf_py = [ id for id in range( max_tri + 1, max_tri + nnewt + 1 )]
    newf = np.array( newf_py )
    newf = np.reshape( newf, ( nnewf, 2 ))
    max_tri = max_tri + nnewt

    ids_py = [ id for id in range( max_face + 1, max_face + nnewf + 1 )]
    ids = np.array( ids_py )
    
    newf = np.c_[ newf, ids ]
    max_face = max_face + nnewf

    faces = np.vstack(( faces, newf ))
    
    _tris = [[ faces[ tri ][ 0 ], faces[ tri ][ 1 ]]  for tri in _faces ]
    _tris = [ item for sublist in _tris for item in sublist ]
    
    # TODO: pra criar os tris novos, tem q determinar as edges deles. Como as edges
    # ja podem existir, possivelmente é melhor continuar o algoritmo e gerar as 
    # novas edges primeiro
    
    _edges = [[ tris[ edge ][ 0 ], tris[ edge ][ 1 ]] for edge in _tris ]
    _edges = [ item for sublist in _edges for item in sublist ]
    print( _edges )

print( faces[ 0 ][ 0:1 ])
print()
subdivide( faces[ 0 ][ 0:1 ], 3 )


[0]

[1, 2, 3, 4]
